In [25]:
import pandas as pd
import nltk
import matplotlib as plt
import sklearn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import gensim
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score

In [26]:
df=pd.read_csv(r'C:\Users\Swarupa V\Desktop\IT-ticket classification\bank_complaint.csv')

In [27]:
df.head()

,ticket,subclass,class,class id
0,I had recently received a check for a settleme...,Problems caused by my funds being low,bank_account,1
1,"At Bank Of America, I have had my personal & b...",Problems caused by my funds being low,bank_account,1
2,"I Bank with Wells fargo, I went in to deposit ...",Problems caused by my funds being low,bank_account,1
3,Charges for my purchases were shown pending on...,Problems caused by my funds being low,bank_account,1
4,I made purchases after my direct deposit was m...,Problems caused by my funds being low,bank_account,1


In [28]:
#function to preprocess text 
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def preprocess(s):
    lst=[]
    s=' '.join(s.split()) #removes extra white spaces+ \n,\t
    s=re.sub(r'[^\w]', ' ', s) #removes symbols
    s=s.lower()#to lower case
    s=[ps.stem(x) for x in word_tokenize(s)]#stem words and tokentise it
    s=[x for x in s if not x in stop_words] #remove stop words
    
    return s

In [34]:
#function to train word2vec model
def w2vmodel(vocab):
    model = gensim.models.Word2Vec(vocab, size=300, min_count=1, workers=4)
    model.train(t, total_examples=len(vocab), epochs=400)
    return model

In [35]:
#function to write the vectors of text into a file
def filewrit(token_lst,wrd2vec_model):
    wrd2vec_filename ='train_review_wrd2vec.csv'
    with open(wrd2vec_filename, 'w+') as wrd2vec_file:
        header = ",".join(str(ele) for ele in range(1022))
        wrd2vec_file.write(header)
        wrd2vec_file.write("\n")
        for row in token_lst:
            line1 = ",".join( [str(vector_element) for vector_element in row] )
            wrd2vec_file.write(line1)
            line2=",".join([str(0) for i in range(1025-len(row))])
            wrd2vec_file.write(line2)
            wrd2vec_file.write('\n')

In [36]:
#calling the preprocess function
complaints=df['ticket']
t=[]
for s in complaints:
    t.append(preprocess(s))

In [37]:
#calling the model function
model=w2vmodel(t)

In [38]:
#obtaining the vectors for each word in a sentance and using its mean to represent the word
import numpy as np
x=[]
x_final=[]
for mail in complaints:
    x=[]
    for word in mail:
        if word in model.wv.vocab:
            x.append(np.mean(model.wv[word]))
        else:
            x.append(0)
    x_final.append(x)
filewrit(x_final,model)

In [57]:
ddf = pd.read_csv(r'train_review_wrd2vec.csv')
x=ddf
y=df['class']
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size = 0.80,random_state=14)

In [58]:
#Descision tree
from sklearn.tree import DecisionTreeClassifier
clf_decision_doc2vec = DecisionTreeClassifier()
clf_decision_doc2vec.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, clf_decision_doc2vec.predict(X_test)))

Final Accuracy: 0.1875


In [59]:
#multi class logistic regression
final_model = LogisticRegression(multi_class='multinomial')
final_model.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(X_test)))

Final Accuracy: 0.2875


In [60]:
#K nearest neighbours
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 4).fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, knn.predict(X_test)))

Final Accuracy: 0.3125


In [62]:
#random forest
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=14)
clf.fit(X_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, clf.predict(X_test)))

Final Accuracy: 0.2875


In [56]:
# Using the word2vec model, we find similarity between 2 words. The model has been trained with the 6000 odd  vocabalary from the dataset
#the model would have performed well with  more data or using a pre-trained model
# KNN gives the best accuracy when training dataset is small while logistic regression for a larger training data